In [41]:
import pandas as pd
import numpy as np
import os
import re
from dotenv import load_dotenv

In [ ]:
# .env 파일 불러오기
load_dotenv()

BASE_NAME = "1-0s"

SYNC_LOG_INPUT_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', 'origin', f'sync_test_log_{BASE_NAME}.csv')
TEGRA_LOG_INPUT_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', 'origin', f'tegra_log_{BASE_NAME}.txt')

SYNC_LOG_OUTPUT_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', f'sync_log_{BASE_NAME}_stable.csv')
TEGRA_LOG_OUTPUT_PATH = os.path.join(os.getenv('BASE_PATH'), 'pbl_exp', 'data', f'tegra_log_{BASE_NAME}_stable.csv')

TRIM_START_SECONDS = 30
STABLE_DURATION_SECONDS = 60 * 10         # 10분간 1초 간격으로 기록 = 600줄

def parse_tegra_line(line, num_cores):
    """tegrastats의 한 줄을 파싱하여 모든 중요 지표를 dict로 반환합니다."""
    data = {}
    
    try:
        # 1. Timestamp (예: "10-29-2025 14:08:39")
        ts_match = re.search(r'^(\d{2}-\d{2}-\d{4}\s\d{2}:\d{2}:\d{2})', line)
        data['Timestamp'] = ts_match.group(1) if ts_match else np.nan

        # 2. RAM/SWAP (예: "RAM 2994/62841MB ... SWAP 0/31420MB")
        ram_match = re.search(r"RAM (\d+)/\d+MB", line)
        data['RAM_Used_MB'] = int(ram_match.group(1)) if ram_match else np.nan
        swap_match = re.search(r"SWAP (\d+)/\d+MB", line)
        data['SWAP_Used_MB'] = int(swap_match.group(1)) if swap_match else np.nan
        
        # 3. GPU (예: "GR3D_FREQ 0%")
        gpu_match = re.search(r"GR3D_FREQ (\d+)%", line)
        data['GPU_Load_%'] = int(gpu_match.group(1)) if gpu_match else np.nan

        # 4. Power (예: "VDD_GPU_SOC 3175mW... VDD_CPU_CV 396mW... VIN_SYS_5V0 4106mW")
        pwr_gpu_match = re.search(r"VDD_GPU_SOC (\d+)mW", line)
        data['Power_GPU_SOC_mW'] = int(pwr_gpu_match.group(1)) if pwr_gpu_match else np.nan
        pwr_cpu_match = re.search(r"VDD_CPU_CV (\d+)mW", line)
        data['Power_CPU_CV_mW'] = int(pwr_cpu_match.group(1)) if pwr_cpu_match else np.nan
        pwr_vin_match = re.search(r"VIN_SYS_5V0 (\d+)mW", line)
        data['Power_VIN_mW'] = int(pwr_vin_match.group(1)) if pwr_vin_match else np.nan

        # 5. Temperatures (예: "cpu@41.781C soc2@37.937C ... tj@41.781C")
        temp_cpu_match = re.search(r"cpu@([\d\.]+)C", line)
        data['Temp_CPU'] = float(temp_cpu_match.group(1)) if temp_cpu_match else np.nan
        temp_tj_match = re.search(r"tj@([\d\.]+)C", line)
        data['Temp_TJ'] = float(temp_tj_match.group(1)) if temp_tj_match else np.nan
        temp_soc0_match = re.search(r"soc0@([\d\.]+)C", line)
        data['Temp_SOC0'] = float(temp_soc0_match.group(1)) if temp_soc0_match else np.nan
        temp_soc1_match = re.search(r"soc1@([\d\.]+)C", line)
        data['Temp_SOC1'] = float(temp_soc1_match.group(1)) if temp_soc1_match else np.nan
        temp_soc2_match = re.search(r"soc2@([\d\.]+)C", line)
        data['Temp_SOC2'] = float(temp_soc2_match.group(1)) if temp_soc2_match else np.nan

        # 6. Individual CPU Cores (예: "CPU [0%@729, 0%@729,...]")
        cpu_match = re.search(r"CPU \[([^\]]+)\]", line)
        if cpu_match:
            core_stats_str = cpu_match.group(1)
            core_stats = core_stats_str.split(',')
            
            # num_cores를 첫 파싱에서 동적으로 결정
            if num_cores[0] is None:
                num_cores[0] = len(core_stats)
            
            for i in range(num_cores[0]):
                core_name_pct = f"CPU{i+1}_%"
                core_name_mhz = f"CPU{i+1}_MHz"
                
                if i < len(core_stats):
                    core_stat = core_stats[i].strip()
                    if core_stat == 'off':
                        data[core_name_pct] = 0
                        data[core_name_mhz] = 0
                    else:
                        core_match = re.search(r"(\d+)%@(\d+)", core_stat)
                        if core_match:
                            data[core_name_pct] = int(core_match.group(1))
                            data[core_name_mhz] = int(core_match.group(2))
                        else: # 파싱 실패시 (드문 경우)
                            data[core_name_pct] = np.nan
                            data[core_name_mhz] = np.nan
                else: # 로그가 잘린 경우
                    data[core_name_pct] = np.nan
                    data[core_name_mhz] = np.nan
        
    except Exception as e:
        print(f"  [경고] Tegra 로그 파싱 중 오류 발생 (무시하고 계속): {e}")
        
    return data


def preprocess_tegra_log():
    """Tegra 로그(txt)를 '상세 파싱'하여 10분 안정화 구간을 CSV로 저장합니다."""
    print(f"\n--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---")

    start_line = TRIM_START_SECONDS
    end_line = start_line + STABLE_DURATION_SECONDS

    try:
        if not os.path.exists(TEGRA_LOG_INPUT_PATH):
            print(f"[오류] {TEGRA_LOG_INPUT_PATH} 파일을 찾을 수 없습니다.")
            return

        with open(TEGRA_LOG_INPUT_PATH, 'r') as f:
            lines = f.readlines()

        original_lines = len(lines)
        
        if original_lines < end_line:
            print(f"[오류] 원본 로그가 너무 짧습니다 ({original_lines}줄). "
                  f"총 {end_line}줄이 필요합니다.")
            return
        
        stable_lines = lines[start_line:end_line]

        parsed_data = []
        num_cores = [None] # [None]을 리스트로 감싸서 참조 전달(pass-by-reference) 흉내
        
        for line in stable_lines:
            if "RAM" in line and "CPU" in line:
                parsed_data.append(parse_tegra_line(line, num_cores))

        if not parsed_data:
            print("[오류] 안정화 구간에서 유효한 로그 라인을 파싱하지 못했습니다.")
            return
            
        df_stable = pd.DataFrame(parsed_data)
        
        # CPU 열들을 맨 뒤로 보내기 (가독성을 위해)
        cpu_cols = [col for col in df_stable.columns if col.startswith('CPU')]
        other_cols = [col for col in df_stable.columns if not col.startswith('CPU')]
        df_stable = df_stable[other_cols + cpu_cols]
        
        df_stable.to_csv(TEGRA_LOG_OUTPUT_PATH, index=False)

        print(f"  [성공] {TEGRA_LOG_INPUT_PATH} ({original_lines}줄)")
        print(f"  -> {TEGRA_LOG_OUTPUT_PATH} ({len(df_stable)}행) CSV 저장 완료.")
        print(f"       (총 {num_cores[0]}개 CPU 코어 정보 포함)")
        print(f"       (안정화 구간: {start_line+1}줄 ~ {end_line}줄)")
    
    except Exception as e:
        print(f"[오류] Tegra 로그 처리 중 예외 발생: {e}")


def preprocess_sync_log():
    """Sync 로그(csv)를 타임스탬프 기준으로 전처리하여 안정화 구간만 저장합니다."""
    print(f"\n--- 'Sync 로그' 전처리 시작 ---")
    try:
        if not os.path.exists(SYNC_LOG_INPUT_PATH):
            print(f"[오류] {SYNC_LOG_INPUT_PATH} 파일을 찾을 수 없습니다.")
            return
            
        df = pd.read_csv(SYNC_LOG_INPUT_PATH)
        
        if df.empty:
            print(f"[오류] {SYNC_LOG_INPUT_PATH} 파일이 비어있습니다.")
            return
            
        original_rows = len(df)
        
        # 타임스탬프 기준으로 정렬
        df = df.sort_values(by='ts_zed').reset_index(drop=True)
        
        # [수정된 로직]
        # 1. 가장 이른 타임스탬프를 찾습니다.
        min_ts = df['ts_zed'].min()
        max_ts = df['ts_zed'].max()
        
        # 2. 안정화 구간의 시작/종료 타임스탬프를 *절대 시간*으로 정의합니다.
        # 예: min_ts=1000 이면, 1030초부터 1630초까지
        stable_start_ts = min_ts + TRIM_START_SECONDS
        stable_end_ts = stable_start_ts + STABLE_DURATION_SECONDS
        
        # 3. 데이터가 10분 30초 분량(stable_end_ts)을 채우는지 확인합니다.
        if max_ts < stable_end_ts:
            print(f"[오류] Sync 로그의 총 재생 시간이 부족합니다.")
            print(f"  안정화 시작({stable_start_ts:.2f}) ~ 종료({stable_end_ts:.2f})가 필요하나,")
            print(f"  데이터는 {max_ts:.2f}초에 끝납니다.")
            return

        # 4. 타임스탬프 기준으로 10분 윈도우를 정확히 필터링합니다.
        df_stable = df[
            (df['ts_zed'] >= stable_start_ts) & 
            (df['ts_zed'] < stable_end_ts)  # < 를 사용하여 600.000초까지 포함
        ].copy()
        
        # 5. 새 파일로 저장
        df_stable.to_csv(SYNC_LOG_OUTPUT_PATH, index=False)
        
        stable_duration = df_stable['ts_zed'].max() - df_stable['ts_zed'].min()
        
        print(f"  [성공] {SYNC_LOG_INPUT_PATH} ({original_rows}행 / {max_ts - min_ts:.2f}초)")
        print(f"  -> {SYNC_LOG_OUTPUT_PATH} ({len(df_stable)}행 / {stable_duration:.2f}초) 저장 완료.")
        print(f"       (안정화 구간: {stable_start_ts:.2f} ~ {stable_end_ts:.2f} 타임스탬프)")

    except Exception as e:
        print(f"[오류] Sync 로그 처리 중 예외 발생: {e}")

In [49]:
print("===========================================")
print("   로그 파일 10분 고정 윈도우 전처리 스크립트   ")
print("   (Tegra TXT -> CSV 파싱 포함)          ")
print("===========================================")
preprocess_tegra_log()
preprocess_sync_log()
print("\n--- 모든 작업 완료 ---")

   로그 파일 10분 고정 윈도우 전처리 스크립트   
   (Tegra TXT -> CSV 파싱 포함)          

--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\tegra_log_0-1s.txt (689줄)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\tegra_log_0-1s_stable.csv (600행) CSV 저장 완료.
       (총 12개 CPU 코어 정보 포함)
       (안정화 구간: 31줄 ~ 630줄)

--- 'Sync 로그' 전처리 시작 ---
[오류] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\sync_test_log_0-1s.csv 파일이 비어있습니다.

--- 모든 작업 완료 ---


In [ ]:
print("===========================================")
print("   로그 파일 10분 고정 윈도우 전처리 스크립트   ")
print("   (Tegra TXT -> CSV 파싱 포함)          ")
print("===========================================")
preprocess_tegra_log()
preprocess_sync_log()
print("\n--- 모든 작업 완료 ---")

   로그 파일 10분 고정 윈도우 전처리 스크립트   
   (Tegra TXT -> CSV 파싱 포함)          

--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\tegra_log_0-5s.txt (674줄)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\tegra_log_0-5s_stable.csv (600행) CSV 저장 완료.
       (총 12개 CPU 코어 정보 포함)
       (안정화 구간: 31줄 ~ 630줄)

--- 'Sync 로그' 전처리 시작 ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\sync_test_log_0-5s.csv (6862행 / 664.47초)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\sync_log_0-5s_stable.csv (6200행 / 599.93초) 저장 완료.
       (안정화 구간: 4231.20 ~ 4831.20 타임스탬프)

--- 모든 작업 완료 ---


In [43]:
print("===========================================")
print("   로그 파일 10분 고정 윈도우 전처리 스크립트   ")
print("   (Tegra TXT -> CSV 파싱 포함)          ")
print("===========================================")
preprocess_tegra_log()
preprocess_sync_log()
print("\n--- 모든 작업 완료 ---")

   로그 파일 10분 고정 윈도우 전처리 스크립트   
   (Tegra TXT -> CSV 파싱 포함)          

--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\tegra_log_1-0s.txt (671줄)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\tegra_log_1-0s_stable.csv (600행) CSV 저장 완료.
       (총 12개 CPU 코어 정보 포함)
       (안정화 구간: 31줄 ~ 630줄)

--- 'Sync 로그' 전처리 시작 ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\sync_test_log_1-0s.csv (9878행 / 658.82초)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\sync_log_1-0s_stable.csv (8999행 / 599.89초) 저장 완료.
       (안정화 구간: 2501.78 ~ 3101.78 타임스탬프)

--- 모든 작업 완료 ---


In [45]:
print("===========================================")
print("   로그 파일 10분 고정 윈도우 전처리 스크립트   ")
print("   (Tegra TXT -> CSV 파싱 포함)          ")
print("===========================================")
preprocess_tegra_log()
preprocess_sync_log()
print("\n--- 모든 작업 완료 ---")

   로그 파일 10분 고정 윈도우 전처리 스크립트   
   (Tegra TXT -> CSV 파싱 포함)          

--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\tegra_log_2-0s.txt (685줄)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\tegra_log_2-0s_stable.csv (600행) CSV 저장 완료.
       (총 12개 CPU 코어 정보 포함)
       (안정화 구간: 31줄 ~ 630줄)

--- 'Sync 로그' 전처리 시작 ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\sync_test_log_2-0s.csv (9907행 / 668.06초)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\sync_log_2-0s_stable.csv (8891행 / 599.99초) 저장 완료.
       (안정화 구간: 6149.41 ~ 6749.41 타임스탬프)

--- 모든 작업 완료 ---


In [47]:
print("===========================================")
print("   로그 파일 10분 고정 윈도우 전처리 스크립트   ")
print("   (Tegra TXT -> CSV 파싱 포함)          ")
print("===========================================")
preprocess_tegra_log()
preprocess_sync_log()
print("\n--- 모든 작업 완료 ---")

   로그 파일 10분 고정 윈도우 전처리 스크립트   
   (Tegra TXT -> CSV 파싱 포함)          

--- 'Tegra 로그' 전처리 시작 (상세 파싱) ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\tegra_log_3-0s.txt (676줄)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\tegra_log_3-0s_stable.csv (600행) CSV 저장 완료.
       (총 12개 CPU 코어 정보 포함)
       (안정화 구간: 31줄 ~ 630줄)

--- 'Sync 로그' 전처리 시작 ---
  [성공] C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\origin\sync_test_log_3-0s.csv (9958행 / 664.14초)
  -> C:/Users/henho/OneDrive/Desktop/MCT_for_ChamDog\pbl_exp\data\sync_log_3-0s_stable.csv (8999행 / 599.87초) 저장 완료.
       (안정화 구간: 6891.94 ~ 7491.94 타임스탬프)

--- 모든 작업 완료 ---
